# This was part of a failed experiment please see vgg16-auto.ipynb 
### Training the bottleneck features of VGG16 
##### Keras with Tensorflow backend
Based on:
- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html 
- https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069 

### 0. Donwload the data sets and prepare the data
A dataset of cats and dogs from an old kaggle competition:
- https://www.kaggle.com/c/dogs-vs-cats/data

Copy 2000/800 validation/training images in the next folder structure
- data
  - train
    - cats
      - 1000 cats
    - dogs
      - 1000 dogs
  - validation
    - cats
      - 400 cats
    - dogs
      - 400 dogs

In [2]:
img_width, img_height = 150, 150

### 1. Download the trained weights (527Mb)
https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3

Or, use Keras:
```python
from keras import applications
vgg16 = applications.VGG16(include_top=True, weights='imagenet')
vgg16.save('vgg16_with_top.h5')
```

I recommend the direct download.

### 2. Build the VGG16 model and load the weights
We are using **channel-last** configuration for Keras

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D

Using TensorFlow backend.


In [13]:
vgg16 = Sequential()
# Block 1
vgg16.add(ZeroPadding2D((0, 0), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
vgg16.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
vgg16.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
vgg16.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
vgg16.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
vgg16.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
vgg16.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
vgg16.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_29 (ZeroPaddi (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
zero_padding2d_30 (ZeroPaddi (None, 152, 152, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 152, 152, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 76, 76, 64)        0         
_________________________________________________________________
zero_padding2d_31 (ZeroPaddi (None, 78, 78, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 78, 78, 128)       73856     
__________

In [14]:
# vgg16.load_weights('models/vgg/vgg16_weights.h5')
import numpy as np
import h5py

f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(vgg16.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = vgg16.layers[k]

    if layer.__class__.__name__ in ['Conv2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

In [20]:
# we are only going to train the bottleneck features
print("Next layers wont be trained")
for layer in vgg16.layers[:]:
    print(layer.get_config()['name'],)
    layer.trainable = False

Next layers wont be trained
zero_padding2d_29
block1_conv1
zero_padding2d_30
block1_conv2
block1_pool
zero_padding2d_31
block2_conv1
zero_padding2d_32
block2_conv2
block2_pool
zero_padding2d_33
block3_conv1
zero_padding2d_34
block3_conv2
zero_padding2d_35
block3_conv3
block3_pool
zero_padding2d_36
block4_conv1
zero_padding2d_37
block4_conv2
zero_padding2d_38
block4_conv3
block4_pool
zero_padding2d_39
block5_conv1
zero_padding2d_40
block5_conv2
zero_padding2d_41
block5_conv3
block5_pool


In [28]:
# model
# top_model = Sequential()
# top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
# top_model.add(Dense(256, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(1, activation='sigmoid'))

In [24]:
from keras import optimizers
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
vgg16.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

### 3. Load data and get bottleneck features

In [40]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16
top_model_weights_path = 'bottleneck_fc_model.h5'

In [33]:
import time

datagen = ImageDataGenerator(rescale=1. / 255)
generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

start = time.time()
bottleneck_features_train = vgg16.predict_generator(generator, nb_train_samples // batch_size, verbose=1)
print("ellapsed time in seconds:", (time.time()-start))

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

start = time.time()
bottleneck_features_validation = vgg16.predict_generator(generator, nb_validation_samples // batch_size, verbose=1)
print("ellapsed time in seconds:", (time.time()-start))

Found 2000 images belonging to 2 classes.
125/125 [==============================] - 597s   
ellapsed time in seconds: 597.9180190563202
Found 800 images belonging to 2 classes.
50/50 [==============================] - 238s   
ellapsed time in seconds: 238.74046516418457


In [34]:
# optional?
np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

### 4. Train top model

In [36]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

In [39]:
from keras.layers import Dropout, Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

start = time.time()
model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
print("ellapsed time in seconds:", (time.time()-start))

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 16s - loss: 7.6575 - acc: 0.5090 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/50
2000/2000 [==============================] - 15s - loss: 7.3925 - acc: 0.5050 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/50
2000/2000 [==============================] - 15s - loss: 7.2231 - acc: 0.5125 - val_loss: 1.1293 - val_acc: 0.7812
Epoch 4/50
2000/2000 [==============================] - 15s - loss: 6.4652 - acc: 0.5450 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/50
2000/2000 [==============================] - 15s - loss: 4.9388 - acc: 0.6045 - val_loss: 1.8261 - val_acc: 0.6675
Epoch 6/50
2000/2000 [==============================] - 15s - loss: 1.0302 - acc: 0.7105 - val_loss: 0.4967 - val_acc: 0.8150
Epoch 7/50
2000/2000 [==============================] - 15s - loss: 0.6018 - acc: 0.7430 - val_loss: 0.6036 - val_acc: 0.6625
Epoch 8/50
2000/2000 [==============================] - 15s - loss: 0.6

NameError: name 'top_model_weights_path' is not defined

In [42]:
model.save_weights(top_model_weights_path)
top_model = model

### 5. Combine vgg16 + top_model and tune fine it

In [44]:
from keras.models import Model
# top_model has the weights 
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_29_input (Inp (None, 150, 150, 3)       0         
_________________________________________________________________
zero_padding2d_29 (ZeroPaddi (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
zero_padding2d_30 (ZeroPaddi (None, 152, 152, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 152, 152, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 76, 76, 64)        0         
_________________________________________________________________
zero_padding2d_31 (ZeroPaddi (None, 78, 78, 64)        0         
__________

In [45]:
for layer in model.layers[:32]:
    layer.trainable = False
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [68]:
import time
start = time.time()
# fine-tune the model
epochs_modified = 1
output = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs_modified,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)
print("ellapsed time in seconds:", (time.time()-start))
output

Epoch 1/1
125/125 [==============================] - 864s - loss: 0.3617 - acc: 0.8460 - val_loss: 0.4041 - val_acc: 0.8688
ellapsed time in seconds: 864.1313540935516


In [48]:
final_model_weights = "vgg16+top_model-train_2h_weights_only.h5"
model.save_weights(final_model_weights)
final_model = "vgg16+top_model-train2h.h5"
model.save(final_model)

### 6. Do predictions

In [49]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import load_model
# from https://gist.github.com/ragvri/6a28b08b9ad844bc66b90db7d7cebb17
def predict_image_class(model, file, w, h):
    x = load_img(file, target_size=(w, h))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    print(array)
    if array[0][0] == 1:
        print("dog")
    else:
        print("cat")

In [53]:
predictor = load_model(final_model)

In [67]:
predict_image_class(predictor, "data/validation/dogs/dog.12199.jpg", img_width, img_height)
predict_image_class(predictor, "data/validation/cats/cat.12100.jpg", img_width, img_height)

[[ 1.]]
dog
[[ 1.]]
dog
